# First-level GLM and contrast: event_friend_reward vs event_stranger_reward

This notebook discovers the fMRIPrep outputs for subject `1001` in a BIDS dataset, builds a first-level GLM across runs of the `sharedreward` task, and computes the contrast `event_friend_reward > event_stranger_reward`. It uses `nilearn` and `pybids`.

In [ ]:
# Install dependencies (run once)
!pip install --quiet nilearn pybids pandas nibabel matplotlib seaborn

In [ ]:
import os
from glob import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nilearn import plotting
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
import nibabel as nib

# User-editable settings
BIDS_ROOT = '/home/jovyan/neurodesktop-storage/ds004920-main'
SUBJECT = '1001'
TASK = 'sharedreward'
CONTRAST_NAME = 'event_friend_reward > event_stranger_reward'
OUTDIR = os.path.join(BIDS_ROOT, 'derivatives', 'glm_sub-{}'.format(SUBJECT))
os.makedirs(OUTDIR, exist_ok=True)

print('BIDS root:', BIDS_ROOT)

Find preprocessed bold files (fMRIPrep-style) and matching events TSVs for the subject and task. The code is robust to fMRIPrep being in `derivatives/fmriprep` or to preprocessed images being placed beside the raw dataset.

In [ ]:
# helper: find preproc bold files for this subject/task
def find_runs(bids_root, subject, task):
    patterns = [
        os.path.join(bids_root, 'derivatives', 'fmriprep', f'sub-{subject}', 'func', f'sub-{subject}_task-{task}_run-*_space-MNI*_desc-preproc_bold*.nii*'),
        os.path.join(bids_root, f'sub-{subject}', 'func', f'sub-{subject}_task-{task}_run-*_bold*.nii*'),
    ]
    runs = []
    for pat in patterns:
        runs.extend(sorted(glob(pat)))
    return sorted(list(dict.fromkeys(runs)))

bold_paths = find_runs(BIDS_ROOT, SUBJECT, TASK)
print('Found preprocessed BOLD images:')
for b in bold_paths:
    print('  ', b)

# find matching events - we expect events next to raw bolds or in dataset root `sub-*/func`
def find_events_for_bold(bold_path):
    # try same folder, then BIDS root func folder
    dirname = os.path.dirname(bold_path)
    basename = os.path.basename(bold_path)
    # compute a task-run pattern from basename
    parts = basename.split('_')
    taskrun = '_'.join(p for p in parts if p.startswith('task-') or p.startswith('run-'))
    fn1 = os.path.join(dirname, f'sub-{SUBJECT}_{taskrun}_events.tsv')
    if os.path.exists(fn1):
        return fn1
    # fallback search in dataset func folder
    fallback = os.path.join(BIDS_ROOT, f'sub-{SUBJECT}', 'func', f'sub-{SUBJECT}_{taskrun}_events.tsv')
    if os.path.exists(fallback):
        return fallback
    # last resort: search for events matching this task/run anywhere under subject func
    globpat = os.path.join(BIDS_ROOT, f'sub-{SUBJECT}', 'func', f'*task-{TASK}*run-*.tsv')
    hits = sorted(glob(globpat))
    if hits:
        return hits[0]
    return None

events_paths = []
for b in bold_paths:
    ev = find_events_for_bold(b)
    events_paths.append(ev)
    print('BOLD:', b)
    print('  events:', ev)

Load events, keep only `event_` trial types (experimental events), and fit a first-level GLM across runs. The design will include one regressor per unique `trial_type`.

In [ ]:
# prepare lists for nilearn (scans and events per run)
scans = []
events_list = []
trs = []
for b, evf in zip(bold_paths, events_paths):
    scans.append(b)
    if evf is None:
        raise RuntimeError(f'No events file found for {b}')
    df = pd.read_csv(evf, sep='	')
    # keep only rows where trial_type starts with 'event_'
    if 'trial_type' not in df.columns:
        raise RuntimeError(f'events file {evf} missing required column `trial_type`')
    df = df[df['trial_type'].str.startswith('event_')].copy()
    # nilearn expects columns onset,duration,trial_type
    df = df[['onset', 'duration', 'trial_type']].reset_index(drop=True)
    events_list.append(df)
    # read TR from sidecar json if available
    json_candidates = [b.replace('.nii.gz', '.json').replace('.nii', '.json')]
    tr = None
    for jc in json_candidates:
        if os.path.exists(jc):
            with open(jc) as fh:
                info = json.load(fh)
                tr = info.get('RepetitionTime', None)
                break
    if tr is None:
        # fallback: try to read TR from nifti header
        img = nib.load(b)
        hdr = img.header
        tr = float(hdr.get_zooms()[3]) if len(hdr.get_zooms()) > 3 else None
    trs.append(tr)

print('TRs per run:', trs)
if len(set([t for t in trs if t is not None])) > 1:
    print('Warning: TR varies across runs. FirstLevelModel will be given the run-specific TRs where possible.')

# instantiate FirstLevelModel. We pass t_r=None and will build design matrices per run automatically by providing events list.
flm = FirstLevelModel(t_r=min([t for t in trs if t is not None]) if any(trs) else 2.0,
                     slice_time_ref=0.5,
                     hrf_model='spm',
                     drift_model='cosine',
                     high_pass=0.01,
                     smoothing_fwhm=6.0,
                     standardize=False,
                     memory='nilearn_cache')

# fit across runs by providing list of runs and events list
flm = flm.fit(run_imgs=scans, events=events_list)
print('Model fit complete')

Compute the contrast `event_friend_reward - event_stranger_reward` and save a z-map NIfTI to `OUTDIR`. Visualize the result.

In [ ]:
# Define contrast using exact condition names found in design columns
design_mtx = flm.design_matrices_[0]  # design matrix for first run (column names are consistent)
print('Design columns:', list(design_mtx.columns))
# expected column names like 'event_friend_reward' and 'event_stranger_reward'
cname_pos = 'event_friend_reward'
cname_neg = 'event_stranger_reward'
if cname_pos not in design_mtx.columns or cname_neg not in design_mtx.columns:
    raise RuntimeError(f'Could not find required regressors in design matrix: {cname_pos}, {cname_neg}')
# build contrast vector
contrast_vec = np.array([1.0 if c == cname_pos else -1.0 if c == cname_neg else 0.0 for c in design_mtx.columns])
z_map = flm.compute_contrast(contrast_vec, output_type='z_score')
out_z = os.path.join(OUTDIR, f'sub-{SUBJECT}_task-{TASK}_contrast-friend_vs_stranger_zmap.nii.gz')
z_map.to_filename(out_z)
print('Saved z-map to', out_z)

# Plot a glass brain and a cut-surface plot
display = plotting.plot_glass_brain(z_map, threshold=3.1, display_mode='ortho', colorbar=True, title=CONTRAST_NAME)
plt.show()

# also create a quick orthoview (may be slow)
plotting.plot_stat_map(z_map, threshold=3.1, display_mode='z', cut_coords=7, title=CONTRAST_NAME)
plt.show()

Notes and next steps:
- This is a single-subject first-level analysis that models the `event_*` trial types.
- If you want run-level fixed-effects combination or a proper second-level group analysis, compute contrasts for each run separately and combine at the subject- or group-level.
- Adjust high-pass, smoothing, and confound regressors as needed (you can pass confounds per run through the `events` and `confounds` arguments).